In [2]:
import pandas as pd

Read in the xfl_column.csv file.  
We need to specify the encoding type because pandas doesn't understand it as a utf-8 encoded file

In [83]:
xfl_column = pd.read_csv('csv_data/xfl_column.csv', index_col=0, encoding='latin1')

Drop any rows where the description is null. This is required for the regex functions we will use later.

In [86]:
xfl_column.dropna(subset=['description'], inplace=True)

### Generate co_afnd_fd
This will perform the same SQL query as the R code below:
```R
mycols_fd <- match( sqlQuery(ch, stringsAsFactors = F , query = "SELECT columnname FROM xfl_column WHERE tablename IN ('co_afnd1', 'co_afnd2') AND adj_method = '/' AND CHARINDEX('dil',description) > 0 ORDER BY tablename, colnum")[,] , colnames(mydata_af.0) )
```

In [87]:
co_afnd_fd = xfl_column[
    xfl_column['tablename'].isin(['co_afnd1', 'co_afnd2'])
    & xfl_column['description'].str.contains(r'[Dd]ilute', regex=True)
    & (xfl_column['adj_method'] == '/')
].sort_values(['tablename', 'colnum'])

### Generate co_afnd_pr
Performs the SQL query:
```R
mycols_pr <- match( sqlQuery(ch, stringsAsFactors = F , query = "SELECT columnname FROM xfl_column WHERE tablename IN ('co_afnd1', 'co_afnd2') AND adj_method = '/' AND (CHARINDEX('basic',description) > 0 OR columnname IN ('opeps', 'bkvlps') ) ORDER BY tablename, colnum") [,] , colnames(mydata_af.0))
```

In [89]:
co_afnd_pr = xfl_column[
    xfl_column['tablename'].isin(['co_afnd1', 'co_afnd2'])
    & (xfl_column['adj_method'] == '/')
    & (
        xfl_column['description'].str.contains(r'[Bb]asic', regex=True)
        | xfl_column['columnname'].isin(['opeps', 'bkvlps'])
      )
].sort_values(['tablename', 'colnum'])

### Generate co_afnd_unsure
Performs the SQL query:
```R
myunsure_ps <- sqlQuery(ch, stringsAsFactors = F , query = "SELECT columnname, description, * FROM xfl_column WHERE tablename IN ('co_afnd1', 'co_afnd2') AND adj_method = '/' AND (CHARINDEX('basic',description) = 0 AND CHARINDEX('dil',description) = 0) ORDER BY tablename, colnum")
```

In [115]:
co_afnd_unsure = xfl_column[
    xfl_column['tablename'].isin(['co_afnd1', 'co_afnd2'])
    & (xfl_column['adj_method'] == '/')
    & xfl_column['description'].str.contains(r'^((?!([Dd]il)|[Bb]asic).)*$', regex=True)
].sort_values(['tablename', 'colnum'])

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
